In [1]:
from automol.pipeline import Pipeline

In [2]:
config_yaml = 'qm9_dataset_test_config.yaml'
pipeline = Pipeline(config_yaml)
pipeline.print_spec()

amount: 10
dataset_class: QM9
dataset_location: dsgdb9nsd.xyz
labels:
- homo
models_to_exclude:
- GaussianProcessRegressor
- MLPRegressor
- LinearRegression
- GradientBoostingRegressor
problem: regression



In [ ]:
#pipeline.data_set.data['smiles'][0:10]

In [ ]:
#pipeline.data_set.data['homo'][0:10]

In [ ]:
#pipeline.data_set.data

In [ ]:
#data=pipeline.data_set.data
#data.boxplot()

In [ ]:
#from pandas.plotting import scatter_matrix
#scatter_matrix(data, alpha=0.2, figsize=(6, 6), diagonal='kde')

In [5]:
#import pandas as pd
#import rdkit.Chem as Chem

#df = pd.DataFrame(pipeline.data_set.data['smiles'], columns=['smiles'])
#pd.DataFrame([Chem.RDKFingerprint(Chem.MolFromSmiles(smi)) for smi in df['smiles']],
#                                                            columns=['fingerprint'])

In [4]:
pipeline.train()

get_models with problem_type: regression and to_exlcude: ['GaussianProcessRegressor', 'MLPRegressor', 'LinearRegression', 'GradientBoostingRegressor']
generate_models with problem_type: regression and model_list: {'RandomForestRegressor'}
generate_model with problem_type: regression and model_name: RandomForestRegressor
acceptable_feature_types with model_name: RandomForestRegressor
acceptable_feature_types model prefix: RandomForest
get_acceptable_features with acceptable_types: {'vector'}
__generatable_features: {'fingerprint', 'rdkit'}
iam: [{'vector'}, {'vector'}]
generated_models: [<automol.models.ModelAbstraction object at 0x7fd7920cda30>, <automol.models.ModelAbstraction object at 0x7fd7920cdb80>]
fit feature type: <class 'numpy.ndarray'> and y type: <class 'pandas.core.frame.DataFrame'>


ValueError: setting an array element with a sequence.